In [1]:
import torch
import torch.optim as optim
import torchvision.transforms as T
from torchvision.models import resnet50
from PIL import Image
import numpy as np

In [2]:
# Load your target image and preprocess it
target_img_path = './grant-headshot.png'  # Update this path
target_img = Image.open(target_img_path).convert('RGB')
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor()
])
target_img_tensor = transform(target_img).unsqueeze(0)  # Add batch dimension

# Load a pre-trained model
model = resnet50(pretrained=True).eval()

# Use the feature extractor part only
feature_extractor = torch.nn.Sequential(*list(model.children())[:-2])

# Create an image to optimize
optimized_img = torch.randn(target_img_tensor.size(), requires_grad=True)

# Optimizer
optimizer = optim.Adam([optimized_img], lr=0.01)

# Loss function
loss_fn = torch.nn.MSELoss()

# Optimize
num_steps = 300  # Adjust the number of steps as needed
for step in range(num_steps):
    optimizer.zero_grad()
    # Ensure the optimized image is within valid pixel range
    optimized_img_clamped = torch.clamp(optimized_img, 0, 1)
    target_features = feature_extractor(target_img_tensor)
    optimized_features = feature_extractor(optimized_img_clamped)
    loss = loss_fn(optimized_features, target_features)
    loss.backward()
    optimizer.step()
    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss.item()}")

# Convert the optimized image to a PIL image for viewing/saving
optimized_img_np = optimized_img_clamped.detach().squeeze().permute(1, 2, 0).numpy()
optimized_img_pil = Image.fromarray((optimized_img_np * 255).astype(np.uint8))
optimized_img_pil.show()  # Or save with optimized_img_pil.save('optimized_image.jpg')


/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Step 0, Loss: 0.5416491627693176
Step 100, Loss: 0.17150121927261353
Step 200, Loss: 0.11609020084142685
